In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm
import pandas as pd
import os

# Load datasets
train_x = pd.read_csv('kaggle_data/train_x.csv')
train_y = pd.read_csv('kaggle_data/train_y.csv')
val_x = pd.read_csv('kaggle_data/val_x.csv')
val_y = pd.read_csv('kaggle_data/val_y.csv')


def preprocess_texts(texts):
    texts = texts.fillna("[UNK]")  # Replace NaN values with a placeholder
    texts = texts.astype(str)  # Ensure all entries are strings
    return texts

train_x['string'] = preprocess_texts(train_x['string'])
val_x['string'] = preprocess_texts(val_x['string'])

# Handle dataset imbalance
def balance_dataset(train_x, train_y):
    # Merge inputs and labels for balancing
    data = pd.concat([train_x, train_y], axis=1)

    # Separate classes
    toxic_data = data[data['y'] == 1]
    non_toxic_data = data[data['y'] == 0]

    # Balance the dataset by sampling the minority class
    if len(toxic_data) > len(non_toxic_data):
        toxic_data = toxic_data.sample(len(non_toxic_data), random_state=42)
    else:
        non_toxic_data = non_toxic_data.sample(len(toxic_data), random_state=42)

    balanced_data = pd.concat([toxic_data, non_toxic_data]).sample(frac=1, random_state=42)

    return balanced_data.drop(columns=['y']), balanced_data

train_x, train_y = balance_dataset(train_x, train_y)

def predict_and_save(test_path, model_path, output_path):
    if not os.path.exists(test_path):
        raise FileNotFoundError(f"The file {test_path} does not exist.")

    test_x = pd.read_csv(test_path)

    if 'string' not in test_x.columns:
        raise ValueError("The test dataset must contain a 'string' column.")

    test_x['string'] = preprocess_texts(test_x['string'])
    test_texts = test_x['string'].tolist()

    # Prepare test dataset
    test_dataset = ToxicCommentDataset(test_texts, [0]*len(test_texts), tokenizer, max_len)  # Dummy labels
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

    # Load the trained model
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"The model file {model_path} does not exist.")
    
    model = ToxicCommentClassifier(n_classes=1)
    model.load_state_dict(torch.load(model_path))
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    # Predict
    test_preds = []
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Predicting", leave=False):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            with torch.amp.autocast('cuda'):
                outputs = model(input_ids, attention_mask)
            test_preds.extend(torch.sigmoid(outputs).cpu().numpy())

    test_preds = [1 if p > 0.5 else 0 for p in test_preds]

    # Save predictions
    test_x['prediction'] = test_preds
    test_x[['index', 'prediction']].to_csv(output_path, index=False)
    print(f"Predictions saved to {output_path}")

# Define the Focal Loss class
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=3, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        BCE_loss = nn.BCEWithLogitsLoss(reduction='none')(inputs, targets)
        pt = torch.exp(-BCE_loss)  # Prevents nans when probability is 0
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduction == 'mean':
            return F_loss.mean()
        elif self.reduction == 'sum':
            return F_loss.sum()
        else:
            return F_loss

# Define a custom Dataset
class ToxicCommentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.float)
        }

# Define the BERT-based model
class ToxicCommentClassifier(nn.Module):
    def __init__(self, n_classes):
        super(ToxicCommentClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = output.pooler_output
        dropped_out = self.dropout(pooled_output)
        return self.out(dropped_out)

# Load data
train_texts = train_x['string'].tolist()
train_labels = train_y['y'].values
val_texts = val_x['string'].tolist()
val_labels = val_y['y'].values

# Tokenizer and dataset
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_len = 128

train_dataset = ToxicCommentDataset(train_texts, train_labels, tokenizer, max_len)
val_dataset = ToxicCommentDataset(val_texts, val_labels, tokenizer, max_len)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Model, loss, and optimizer
model = ToxicCommentClassifier(n_classes=1)
loss_fn = FocalLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
scaler = torch.amp.GradScaler('cuda')  # For mixed precision training

# Define the worst group accuracy function
def worst_group_accuracy(targets, preds, groups):
    group_accuracies = {}

    for group in groups.columns:
        group_mask = groups[group].astype(bool).values  # Ensure group_mask is boolean
        group_targets = targets[group_mask]
        group_preds = preds[group_mask]

        if len(group_targets) > 0:
            accuracy = accuracy_score(group_targets, group_preds)
            group_accuracies[group] = accuracy

    worst_group = min(group_accuracies, key=group_accuracies.get)
    return worst_group, group_accuracies[worst_group]

# Training loop
def train_model(model, train_loader, val_loader, loss_fn, optimizer, epochs=3, model_path="model/best_model.pth", patience=3):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    best_val_loss = float('inf')
    no_improve_epochs = 0

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        print(f"Epoch {epoch+1}/{epochs}")
        for batch in tqdm(train_loader, desc="Training", leave=False):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()

            with torch.amp.autocast('cuda'):
                outputs = model(input_ids, attention_mask)
                loss = loss_fn(outputs.squeeze(-1), labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            total_loss += loss.item()

        print(f"Loss: {total_loss / len(train_loader):.4f}")

        # Validation
        model.eval()
        val_preds = []
        val_targets = []

        with torch.no_grad():
            for batch in tqdm(val_loader, desc="Validating", leave=False):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                with torch.amp.autocast('cuda'):
                    outputs = model(input_ids, attention_mask)
                val_preds.extend(torch.sigmoid(outputs).cpu().numpy())
                val_targets.extend(labels.cpu().numpy())

        val_preds = [1 if p > 0.5 else 0 for p in val_preds]
        val_loss = loss_fn(torch.tensor(val_preds, dtype=torch.float), torch.tensor(val_targets, dtype=torch.float)).item()

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            no_improve_epochs = 0
            torch.save(model.state_dict(), model_path)
            print(f"Model saved to {model_path}")
        else:
            no_improve_epochs += 1
            if no_improve_epochs >= patience:
                print("Early stopping triggered.")
                break

        print("Validation Accuracy:", accuracy_score(val_targets, val_preds))
        print(classification_report(val_targets, val_preds))


train_model(model, train_loader, val_loader, loss_fn, optimizer, epochs=5, model_path="model/best_model.pth", patience=3)

# Predict and save predictions
predict_and_save(test_path='kaggle_data/test_x.csv', model_path="model/best_model.pth", output_path="prediction.csv")


Epoch 1/5


Loss: 0.0505


Model saved to model/best_model.pth
Validation Accuracy: 0.8480301018149624
              precision    recall  f1-score   support

         0.0       0.98      0.84      0.91     40125
         1.0       0.42      0.88      0.57      5055

    accuracy                           0.85     45180
   macro avg       0.70      0.86      0.74     45180
weighted avg       0.92      0.85      0.87     45180

Epoch 2/5


Loss: 0.0378


Validation Accuracy: 0.8461487383798141
              precision    recall  f1-score   support

         0.0       0.98      0.84      0.91     40125
         1.0       0.41      0.89      0.56      5055

    accuracy                           0.85     45180
   macro avg       0.70      0.87      0.74     45180
weighted avg       0.92      0.85      0.87     45180

Epoch 3/5


...

Validation Accuracy: 0.8196104471004869
              precision    recall  f1-score   support

         0.0       0.98      0.81      0.89     40125
         1.0       0.37      0.90      0.53      5055

    accuracy                           0.82     45180
   macro avg       0.68      0.85      0.71     45180
weighted avg       0.92      0.82      0.85     45180

Epoch 5/5


Loss: 0.0104


Model saved to model/best_model.pth
Validation Accuracy: 0.8552456839309429
              precision    recall  f1-score   support

         0.0       0.98      0.86      0.91     40125
         1.0       0.43      0.85      0.57      5055

    accuracy                           0.86     45180
   macro avg       0.70      0.85      0.74     45180
weighted avg       0.92      0.86      0.87     45180



C:\Users\ismoi\AppData\Local\Temp\ipykernel_32716\2228154450.py:68: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


Predictions saved to prediction.csv


In [8]:
train_labels.shape

(61030,)